# Pruebas tamaños de ventana

Lo que se pretende en este notebook de python en entrenar un modelo RandomForest y posteriormente uno SGDRegressor con distintos tamaños de ventana, teniendo en cuenta que cada muestra se toma cada minutos, la ventana será inicialmente de 5 muestras (25 minutos) y se predirá la siguiente muestra (siguientes 5 minutos), después las 10 siguientes muestras y de nuevo los siguientes 5 minutos, y así sucesivamente. se ha decidido probar con los tamaños que se van a mostrar a continuación.
![](https://drive.google.com/uc?export=view&id=1QhZFSbntE-EYPzfZz1S_0YmaCTwt8r2F)





Lo primero que tedremos que hacer será instalar todos aquellos paqutes necesarios para la ejecución del código.

In [ ]:
!pip install skops
!pip install matplotlib
!pip install pandas


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.2/112.2 kB 684.6 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 10.1 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pickle
import sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import SGDRegressor
import matplotlib.pyplot as plt
from sklearn.metrics import make_scorer
from sklearn.metrics import mean_absolute_percentage_error,mean_absolute_error
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold, cross_val_score
from sklearn import datasets
import pandas as pd
from sklearn.model_selection import TimeSeriesSplit
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn import preprocessing
from sklearn.metrics import mean_squared_error
import os
import skops.io as io
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.multioutput import MultiOutputRegressor




In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


# Random Forest

Estas configuraciones de ventana se harán en primer lugar para el modelo RandomForest.
Random Forest es una de los algoritmos más populares y más comunmente usados por científicos del Dato. Random forest es un algorítmo de Machine Learning supervisado que es ampliamente utilizado y problemas de regresión y clasificación. Este crea árboles de decisión a partir de diferentes muestras y toma su voto mayoritario para clasificación y su aproximación para regresión.

Una de las características más importantes del algoritmo Random Forest es que puede soportar data sets con variables contínuas, como es el caso de la regresión, y variables categóricas como es el caso de la clasificación.








El sobreajuste suele ser un problema crítico que puede empeorar los resultafos obtenidos pero para el algortimo random forest, si hay suficientes árboles en el bosque el casificador con reajustará el modelo. Otro punto a favor de este algoritmo es que puede soportar valores vacíos, no obstante para nuestro ejemplo no trataremos con ellos pues como se muestra en el jupyter [Transformación de Datos](https://colab.research.google.com/drive/13Bu4aIENRpZTCj9K0ozJAoSONLPTK5jI?usp=sharing), se han reconstruido todos aquellos tramos horarios en los que había cortes de luz y se perdían muetras.

# Cómo funciona este algoritmo
Para explicarlo nos basaremos en la siguiente ilustación. Como se observa en la imagen inferior observamos como a partir de los dígitos iniciales que vienen subrayados y en distintos colores el árbol va tomando decisiones y categorizándolo. Tenemos el siguiente número 1 1 0 0 0 0 0, que presenta distintos colores y algunos de ellos están subrayados y otros no. Queremos categorizar dicho número en: rojo, azul y subrayado. El funcionamiento de un árbol de decisión viene definido por un esquema, la primera pregunta clasificará a aquellos números que sean de color rojo por una lado y los de otro color por otro, para la siguiente clasificación solo se tiene en cuenta el conjunto de la izquierda pues el de la derecha ya ha llegado a una categorá final: azul. Siguiendo con el conjunto de la izquierda podemos categorizar en subrayado y no subrayado, por tanto tendremos a la izquierda los unos subrayados y a la derecha el cero sin subrayar. Para nuestro problema es algo más complejo pero entendiendo las bases se puede extrapolar perfectamente a casos de mayor envergadura.


![](https://drive.google.com/uc?export=view&id=1hoYiU6y5bpVt6yS51cdKbHUfxiDzMBP4)



Pero en este caso no estaríamos hablando de un único árbol de decisión si no de muchos árboles que conformarán 'El bosque', de ahí el nombre del algoritmo. Estos arbolitos trabajan como un conjunto y este comportamiento es la clave de su  baja correlación y en consecuencia su mejor rendimiento frente a algorítmos con una alta correlación. Cada árbol da una predicción de forma que el impacto de la predicción se divide entre el número de árboles totales, de forma que la predicción final está protegida de los pequeños errores individuales. Uso por tanto la sabiduría de masas.


# Transformar serie temporal en ventanas

Iremos modificando los tamaños de ventana y comprobando que resultados se van obteniendo tras entrenar el modelo con la mejor configuración de validación cruzada. Primero tomaremos con pickle la serie que contiene todas las series.


In [ ]:
with open('/content/gdrive/MyDrive/PatronesComportamiento/mySeriesInten.pkl', 'rb') as file:
    timeSeries = pickle.load(file)

with open('/content/gdrive/MyDrive/PatronesComportamiento/singleSerieIntenTrT.pkl', 'rb') as file:
    singleSerieTrT = pickle.load(file)


with open('/content/gdrive/MyDrive/PatronesComportamiento/singleSerieIntenValit.pkl', 'rb') as file:
    singleSerieValit= pickle.load(file)

print((singleSerieTrT[:10]))
print(len(timeSeries[0]))



[1.288288, 1.207547, 1.316038, 1.264151, 1.132075, 0.8962264, 1.042453, 0.6650943, 0.8207547, 0.9009434]
287


Lo primero que vamos a hacer es escalar los datos con sk learn en su paquete [Preprocesing](https://scikit-learn.org/stable/modules/preprocessing.html#standardization-or-mean-removal-and-variance-scaling) tiene alguns métodos que permiten escalar los datos de entrenamiento según distintos criterios.

In [ ]:
def scale(data):  
  min_max_scaler = preprocessing.MinMaxScaler()
  X_train=np.array(data)
  X_train = X_train.reshape((len(data), 1))
  X_train_minmax = min_max_scaler.fit_transform(X_train)
  X_train_minmax=[ele[0] for ele in X_train_minmax]
  return X_train_minmax


En primer lugar aplicaremos una ventana a los datos, en este caso la ventana será de 5 y la h de 1. La forma que hemos elegido para aplicar la ventana deslizante es a través de sliding vectorial, posteriomente se añade a un dataframe de pandas. A continuación en la celda contigua podemos ver la definición del método que aplicará la ventana deslizante. Para entender el por qué de utilizar esta técnica ir al jupyter [Transformar datos para prediccion](https://colab.research.google.com/drive/1HuCj1Cetas3ffaXV8q83ynGQaV_njtI6?usp=sharing).


In [ ]:
def spliting_timeseries(timeSerie,w,h):  #w amplitud de la ventana y h >1 si es multiobjetivo
  X, y = [], []  #inicializamos los arrays que guardarán atributos y variable predicha
  for i in range(len(timeSerie) - w): #bucle para iterar sobre todas las muestras de las series 
    X.append(timeSerie[i : i + w])  #cogemos desde el elemento en el que estamos hasta los w siguientes
    y.append(timeSerie[i + w:i+w+h])  #aquí cogemos desde el w+1 hasta los h siguientes
  return pd.DataFrame(X), pd.Series(y),X,y



Lo siguiente será llamar al método con aquellos parámetros con los que queramos configurar la ventana deslizante.

In [ ]:
def prepare_data(singleSerieTrT,w,h):
  X_train_minmax=scale(singleSerieTrT)
  cond,consec=[],[]
  x,y,x_train,y_train=spliting_timeseries(X_train_minmax,5,1) #llamo al método
  cond.append(x)  #lo añado a una lista de atributos
    # lo añado a una lista de variables predichas



  data_cond = pd.concat(cond, axis=0) # combino todos los DataFrames en uno solo
  data_consec = pd.concat([y], axis=0) # combino todos los DataFrames en uno solo
  return x,y,x_train,y_train


El siguiente paso a abordar será dividir los datos en entrenamiento y test. Algo que debemos tener en cuenta a la hora de hacer esta división es que nuestro objeto de estudio son series temporales, por tanto debemos plantear una for de dividir los datos sin que afecte a la línea temporal de los mismos. (Explicar con detalle cuando lea información al respecto).
Para que la división utilizada respete la cronología se utilizará el método: [TimeSeriesSplit](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.TimeSeriesSplit.html#sklearn.model_selection.TimeSeriesSplit) de la librería SK Learn. Este método permite aplicar validación cruzada a series temporales respectando la cronología, Para este método si k es el número de pligues devolverá el pliegue k como el último conjunto que formaŕá parte del conjunto de entrenamiento y el k+1 como el primero del conjunto de test. A diferencia de los métodos cotidianos de validación cruzada los conjuntos sucesivos de entrenamiento son superconjuntos de los conjuntos de entrenamiento previos. Através de este método obtendrems los índices de los conjuntos que formarán parte del entrenamiento y de los que formarán parte del test.
De entre los dintintos parámtros configurables en le método tan sólo configuraremos n_splits a 5, es meramente informativo pues es el valor por defecto del parámetro.


In [ ]:
s=5
tscv = TimeSeriesSplit(n_splits=s)
print(tscv)

fols_dict={}

for i, (train_index, test_index) in enumerate(tscv.split(data_cond)):
    print(f"Fold {i}:")

    fols_dict[i]=[train_index,test_index]
    
    print(f"  Train: index={train_index}")
    print(f"  Test:  index={test_index}")
print(data_cond.iloc[0])

TimeSeriesSplit(gap=0, max_train_size=None, n_splits=5, test_size=None)
Fold 0:
  Train: index=[    0     1     2 ... 29400 29401 29402]
  Test:  index=[29403 29404 29405 ... 58802 58803 58804]
Fold 1:
  Train: index=[    0     1     2 ... 58802 58803 58804]
  Test:  index=[58805 58806 58807 ... 88204 88205 88206]
Fold 2:
  Train: index=[    0     1     2 ... 88204 88205 88206]
  Test:  index=[ 88207  88208  88209 ... 117606 117607 117608]
Fold 3:
  Train: index=[     0      1      2 ... 117606 117607 117608]
  Test:  index=[117609 117610 117611 ... 147008 147009 147010]
Fold 4:
  Train: index=[     0      1      2 ... 147008 147009 147010]
  Test:  index=[147011 147012 147013 ... 176410 176411 176412]
0    0.025950
1    0.025034
2    0.026264
3    0.025676
4    0.024178
Name: 0, dtype: float64


Ahora tenemos distintos conjuntos de entrenamiento y test, de cara al siguiente paso tendremos que tener elegida una configuaración, para ello calcularemos el score que se obtiene para cada una de ellas y aquel que consiga un mayor valor será selecionado. No obstante, aún quedan configuraciones que hacer antes de conseguir el conjunto de entrenamiento y test así como el modelo final.
Tenemos que configurar los distintos conjuntos de entrenamiento y test y probar su rendimiento con la fución score. 

In [ ]:
train_x=[]
test_x=[]
train_y=[]
test_y=[]
splits=tscv.split(data_cond)
for i, (train_index, test_index) in enumerate(tscv.split(data_cond)):
  print(train_index)
  train_x.append(data_cond.iloc[train_index])
  test_x.append(data_cond.iloc[test_index])
  train_y.append(data_consec.iloc[train_index])
  test_y.append(data_consec.iloc[test_index])

print(train_y[0])
for i in range(s):
  train_y[i] = train_y[i].apply(lambda x: float(x[0]))
  test_y[i] = test_y[i].apply(lambda x: float(x[0]))



print(train_y[1])

[    0     1     2 ... 29400 29401 29402]
[    0     1     2 ... 58802 58803 58804]
[    0     1     2 ... 88204 88205 88206]
[     0      1      2 ... 117606 117607 117608]
[     0      1      2 ... 147008 147009 147010]
0        [0.021503597907142312]
1        [0.023161837666766234]
2        [0.018882512291082218]
3        [0.020647733285613742]
4        [0.021557089658616704]
                  ...          
29398       [0.340317312570777]
29399     [0.34613744647139283]
29400      [0.3459133640017863]
29401      [0.3387979517773618]
29402      [0.3514722195572738]
Length: 29403, dtype: object
0        0.021504
1        0.023162
2        0.018883
3        0.020648
4        0.021557
           ...   
58800    0.349949
58801    0.339208
58802    0.300955
58803    0.317366
58804    0.287858
Length: 58805, dtype: float64


Ahora calculamos el score para cada uno de los conjuntos de cross validation. Como se observa a continuación los resultados obtennidos son muy similares dl orden de milésimas, pero escogeremos de todos modos el segundo que es algo mejor que el resto.

A continuación, pasamos a hacer la rejilla para calcular los hiperparámetros óptimos. Para este paso era necesario deteerminar en primer lugar cuáles sería los parámetros que optarían a ser parte de la configuración de nuestro modelo. De entre los parám

Ahora vamos a representar los resultados obtenidos para comprobar la evolución del error MAPE con los distintos parámetros. El mejor error MAPE será aquel que sea igual a 0, por tanto estamos buscando que el error sea en valor absoluto lo más cercano a 0 posible. La razón por la que tiene que ser en valor absoluto es porque el scoring que estamos usando de sklearn da los resultados del error MAPE (entre otros) en negativo.
Vamos a representar el número de estimadores frente a la puntuación obtenida con MAPE. Es cierto que para cada punto dibujado en la gráfica si tiene en cuenta un criterio distinto, por tanto tendremos que localizar para que pareja criterio-número_de_estimadores se optiene un mejor valor.


Obtenemos el mejor valor para la siguientes configuraciones, para saber que configuración nos da los mejores resultados (aclarar que los resultados como se ven están bastante ajustados). Se propone para averiguarlo calcular la diferencia de cada par entrenamiento-test y aquella diferencia que sea la mejor de todas será la elegida como la mejor. Esto podemos permitirnoslo pues en general los resultados oscilan a el mismo valor con el orden de centésimas en la mayoría de casos.
Se podría caer en pensar que est clase de métrica no evitaría el sobreajuste pero es cierto que si de por si para ambos conjuntos (entrenamiento y test) se consiguen valores bajos y similares querá decir que hay un buen rendimiento tanto durante el entrenamiento como durante las pruebas.
Por ello vamos a calcular las diferencias y posteriomente selecionar a un ganador.

In [ ]:
ranking=[]
for r in range(len(cv_results["mean_train_score"])):
  ranking.append(abs(cv_results["mean_train_score"][r]-cv_results["mean_test_score"][r]))
print(ranking)
print(ranking.index(min(ranking)))

Al usar Grid Search nos devuelve un modelo ya entrenado con la mejor configuración, por tanto, ahora probaremos con distintas configuraciones de ventanas para ver calidad de las predicciones.

Calculamos el MAE:

In [ ]:
#mean_squared_error(y_test, y_predicted)
lista=list(range(0,5)[0:3])
i=10
f=5
with open('/content/gdrive/MyDrive/PatronesComportamiento/fotosRandomForest/gridSearchmodelRandomForestw'+str(i)+'f'+str(s)+'.pkl', 'wb') as file:
        pickle.dump(lista, file) 

Ahora vamos a probar con distintos tamaños de ventana que error obtenemos, para ello, una buena práctica sería ir guardando en ficheros separados todos los modelos entrenados, posteriormente calcularíamos su error, este proceso va a tardar mucho en ejecutar pues habría que entrenar un moelo por cada variación en la configuración.

In [ ]:
p=10
s=5
tscv = TimeSeriesSplit(n_splits=s)
regr = RandomForestRegressor(max_depth=5, random_state=0)

# step-1: create a cross-validation scheme

# step-2: specify range of hyperparameters to tune
hyper_params = {'n_estimators': [50,150,200],'criterion':['squared_error','friedman_mse']}


# step-3: perform grid search
# 3.1 specify model

# 3.2 call GridSearchCV()
model_cv = GridSearchCV(estimator = regr, 
                        param_grid = hyper_params, 
                        scoring= 'neg_mean_squared_error', 
                        cv = tscv, 
                        verbose = 1,
                        return_train_score=True)      

# fit the model
files=[]
for i in range(60,288,p):
  print("======Ahora mismo la ventana es : "+str(i)+"=================")

  print("======Ahora mismo el salto es : "+str(p)+"===================")
  X_train_minmax=scale(singleSerieTrT)
  x,y,x_train,y_train=spliting_timeseries(X_train_minmax,i,1) 

  y_train=y_train[0:len(x_train)]
  y_train=[l[0] for l in y_train]
  

  model_cv.fit(x_train,y_train)

    
  io.dump(model_cv, 'fotosRandomForest/files/VentanagridSearchmodelRandomForestw'+str(i)+'f'+str(s)+'.skops')
  

  if i==50:
    p=20  
  elif i==100:
    p= 30
  elif i==280:
    p=7
  
    





======Ahora mismo la ventana es : 60=================
======Ahora mismo el salto es : 10===================
Fitting 5 folds for each of 6 candidates, totalling 30 fits


In [ ]:
directory='/content/gdrive/MyDrive/PatronesComportamiento/fotosRandomForest/files'
files=os.listdir(directory)
for file in files:
    
    model_cv=io.load(directory+'/'+file, trusted=True)
    name_split=file.split('w')
    w=name_split[1].split('f')[0]
    f=name_split[1].split('f')[0].split('.skops')[0]

    print("Ventana: "+w)
    X_test_minmax=scale(singleSerieValit)
    x,y,x_test,y_test=spliting_timeseries(X_test_minmax,int(w),1) #llamo al método
    y_predicted=model_cv.predict(x_test)    
    print(mean_absolute_error(y_test, y_predicted))



Ventana: 240
0.06514649583432994
Ventana: 10
0.06446690579073268
Ventana: 270
0.06522446610833704
Ventana: 50
0.0645312776663096
Ventana: 90
0.06465631287320467
Ventana: 40
0.06455158775640803
Ventana: 288
0.06531306357822554
Ventana: 100
0.06475366269641983
Ventana: 120
0.06477284412094883
Ventana: 70
0.06454804145508791
Ventana: 140
0.06486281464337555
Ventana: 180
0.06492949053019263
Ventana: 130
0.06472420528595953
Ventana: 20
0.06452401165653562
Ventana: 150
0.06467707689119896
Ventana: 60
0.06462280273435314
Ventana: 110
0.06470686043996093
Ventana: 80
0.06465513821663608
Ventana: 210
0.0650434565900811
Ventana: 30
0.06452679745099377


Ahora las predicciones las haremos para un mayor valor de h se forma que vamos a predecir más muestras. Usaremos a librería MultiOutputRegressor de sklearn.

In [ ]:

p=10
s=5
tscv = TimeSeriesSplit(n_splits=s)
regr = RandomForestRegressor(max_depth=5, random_state=0)

# step-1: create a cross-validation scheme

# step-2: specify range of hyperparameters to tune
hyper_params = {'n_estimators': [50,150,200],'criterion':['squared_error','friedman_mse']}


# step-3: perform grid search
# 3.1 specify model

# 3.2 call GridSearchCV()
clf = MultiOutputRegressor(regr)
model_cv = GridSearchCV(estimator = regr, 
                        param_grid = hyper_params, 
                        scoring= 'neg_mean_squared_error', 
                        cv = tscv, 
                        verbose = 1,
                        return_train_score=True)      

# fit the model
files=[]
wh=[[144,144],[288,288],[288,144]]
for pack in wh:
  print("======Ahora mismo la ventana es : "+str(pack[0])+"=================")

  print("======Ahora mismo el salto es : "+str(pack[1])+"===================")
  X_train_minmax=scale(singleSerieTrT)
  x,y,x_train,y_train=spliting_timeseries(X_train_minmax,pack[0],pack[1]) 

  y_train=y_train[0:len(x_train)]
  y_train=[l[0] for l in y_train]
  

  model_cv.fit(x_train,y_train)

    
  io.dump(model_cv, 'fotosRandomForest/files/VentanagridSearchmodelRandomForestw'+str(i)+'f'+str(s)+'.skops')
  



======Ahora mismo la ventana es : 144=================
======Ahora mismo el salto es : 144===================
Fitting 5 folds for each of 6 candidates, totalling 30 fits


KeyboardInterrupt: ignored

# SGD Regressor

Ahora tendremos que hacer lo mismo para este otro modelo, en pruebas anteriores que se pueden encontrar en el notebook [SGDREgressor](https://colab.research.google.com/drive/1RTkMuPEW5UMJ0gCuaGMnELhT0u1TYRUq?usp=sharing). Lo métodos y están definidos, ahora sólo hay que entrenar los modelos para este algoritmo.

In [ ]:
p=10

s=5
tscv = TimeSeriesSplit(n_splits=s)
regr =  SGDRegressor(max_iter=1000, tol=1e-3)

# step-1: create a cross-validation scheme

# step-2: specify range of hyperparameters to tune
hyper_params = {'loss': ['squared_error','huber','epsilon_insensitive'],'penalty':['elasticnet','l1']}


# step-3: perform grid search
# 3.1 specify model

# 3.2 call GridSearchCV()
model_cv = GridSearchCV(estimator = regr, 
                        param_grid = hyper_params, 
                        scoring= 'neg_mean_squared_error', 
                        cv = tscv, 
                        verbose = 1,
                        return_train_score=True)      

# fit the model
files=[]
sizes=[10,20,30,40,50,70,90,110,140,170,200,230,260,288]
for i in sizes:
  print("======Ahora mismo la ventana es : "+str(i)+"=================")

  X_train_minmax=scale(singleSerieTrT)
  x,y,x_train,y_train=spliting_timeseries(X_train_minmax,i,1) 

  y_train=y_train[0:len(x_train)]
  y_train=[l[0] for l in y_train]
  

  model_cv.fit(x_train,y_train)

    
  io.dump(model_cv, '/content/gdrive/MyDrive/PatronesComportamiento/filesSGD/VentanagridSearchmodelSGDRegressorw'+str(i)+'f'+str(s)+'.skops')
  

    


======Ahora mismo la ventana es : 10=================
Fitting 5 folds for each of 6 candidates, totalling 30 fits
======Ahora mismo la ventana es : 20=================
Fitting 5 folds for each of 6 candidates, totalling 30 fits
======Ahora mismo la ventana es : 30=================
Fitting 5 folds for each of 6 candidates, totalling 30 fits
======Ahora mismo la ventana es : 40=================
Fitting 5 folds for each of 6 candidates, totalling 30 fits
======Ahora mismo la ventana es : 50=================
Fitting 5 folds for each of 6 candidates, totalling 30 fits
======Ahora mismo la ventana es : 70=================
Fitting 5 folds for each of 6 candidates, totalling 30 fits
======Ahora mismo la ventana es : 90=================
Fitting 5 folds for each of 6 candidates, totalling 30 fits
======Ahora mismo la ventana es : 110=================
Fitting 5 folds for each of 6 candidates, totalling 30 fits
======Ahora mismo la ventana es : 140=================
Fitting 5 folds for each of 6 ca

In [ ]:
directory='/content/gdrive/MyDrive/PatronesComportamiento/filesSGD'
files=os.listdir(directory)
for file in files:
    
    model_cv=io.load(directory+'/'+file, trusted=True)
    name_split=file.split('w')
    w=name_split[1].split('f')[0]
    f=name_split[1].split('f')[0].split('.skops')[0]

    print("Ventana: "+w)
    X_test_minmax=scale(singleSerieValit)
    x,y,x_test,y_test=spliting_timeseries(X_test_minmax,int(w),1) #llamo al método
    y_predicted=model_cv.predict(x_test)    
    print(mean_absolute_error(y_test, y_predicted))


Ventana: 200
0.020500522302234946
Ventana: 140
0.021079111825852384
Ventana: 288
0.01875494118552419
Ventana: 230
0.019722425513049046
Ventana: 10
0.01968515876198442
Ventana: 70
0.019784449713479146
Ventana: 90
0.021087580596705065
Ventana: 20
0.020565666053166107
Ventana: 40
0.01973199213741264
Ventana: 110
0.019949519343395637
Ventana: 260
0.019474092797366838
Ventana: 170
0.021531222576135135
Ventana: 30
0.01988129320528043
Ventana: 50
0.020017535474110684



De nuevo tenemos que entrenar los modelos variando el tamaño de la h, pero esta vez con SGDREgressor.

In [ ]:

p=10
s=5
tscv = TimeSeriesSplit(n_splits=s)
regr =  SGDRegressor(max_iter=1000, tol=1e-3)

# step-1: create a cross-validation scheme

# step-2: specify range of hyperparameters to tune
hyper_params = {'loss': ['squared_error','huber','epsilon_insensitive'],'penalty':['elasticnet','l1']}


# step-3: perform grid search
# 3.1 specify model

# 3.2 call GridSearchCV()
clf = MultiOutputRegressor(regr)
model_cv = GridSearchCV(estimator = regr, 
                        param_grid = hyper_params, 
                        scoring= 'neg_mean_squared_error', 
                        cv = tscv, 
                        verbose = 1,
                        return_train_score=True)      

# fit the model
files=[]
wh=[[144,144],[288,288],[288,144]]
for pack in wh:
  print("======Ahora mismo la ventana es : "+str(pack[0])+"=================")

  print("======Ahora mismo el salto es : "+str(pack[1])+"===================")
  X_train_minmax=scale(singleSerieTrT)
  x,y,x_train,y_train=spliting_timeseries(X_train_minmax,pack[0],pack[1]) 

  y_train=y_train[0:len(x_train)]
  y_train=[l[0] for l in y_train]
  

  model_cv.fit(x_train,y_train)

    
  io.dump(model_cv, '/content/gdrive/MyDrive/PatronesComportamiento/filesSGD/VentanagridSearchmodelSGDRegressorw'+str(i)+'f'+str(s)+'.skops')


======Ahora mismo la ventana es : 144=================
======Ahora mismo el salto es : 144===================
Fitting 5 folds for each of 6 candidates, totalling 30 fits


KeyboardInterrupt: ignored

In [ ]:
randomForest=[]


directory='/content/gdrive/MyDrive/PatronesComportamiento/fotosRandomForest/files'
files=os.listdir(directory)
for file in files:
    model_cv=io.load(directory+'/'+file, trusted=True)
    w=file.split('w')[1].split('f')[0]
    f=file.split('f')[1].split('.skops')[0]
    criterion=model_cv.best_params_['criterion']
    n_estimators=model_cv.best_params_['n_estimators']
    X_test_minmax=scale(singleSerieValit)
    x,y,x_test,y_test=spliting_timeseries(X_test_minmax,int(w),1) #llamo al método
    y_predicted=model_cv.predict(x_test)    
    MAE=mean_absolute_error(y_test, y_predicted)
    randomForest.append(['randomForest',w,1,n_estimators,criterion,MAE])
  
randomForest=sorted(randomForest, key=lambda e: e[4])

directory='/content/gdrive/MyDrive/PatronesComportamiento/fotosRandomForest/filesHwide'
files=os.listdir(directory)
for file in files:
    model_cv=io.load(directory+'/'+file, trusted=True)
    w=file.split('w')[1].split('f')[0]
    f=file.split('f')[1].split('.skops')[0]
    criterion=model_cv.best_params_['criterion']
    n_estimators=model_cv.best_params_['n_estimators']
    X_test_minmax=scale(singleSerieValit)
    x,y,x_test,y_test=spliting_timeseries(X_test_minmax,int(w),1) #llamo al método
    y_predicted=model_cv.predict(x_test)    
    MAE=mean_absolute_error(y_test, y_predicted)
    randomForest.append(['randomForest',w,f,n_estimators,criterion,MAE])
  
randomForest=sorted(randomForest, key=lambda e: e[4])
dfRandom = pd.DataFrame(np.array(randomForest),
                   columns=['algoritmo','ventana','h','n_estimators','criterion','MAE'])

/usr/local/lib/python3.9/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator DecisionTreeRegressor from version 1.1.3 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator RandomForestRegressor from version 1.1.3 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator GridSearchCV from version 1.1.3 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own ris

In [ ]:
dfRandom

,algoritmo,ventana,h,n_estimators,criterion,MAE
0,randomForest,240,1,200,friedman_mse,0.0636872970315142
1,randomForest,10,1,150,friedman_mse,0.0637104920738452
2,randomForest,50,1,150,friedman_mse,0.06343909696898647
3,randomForest,90,1,200,friedman_mse,0.06353362579572978
4,randomForest,120,1,200,friedman_mse,0.06364430195428931
5,randomForest,140,1,200,friedman_mse,0.06375649289060083
6,randomForest,180,1,150,friedman_mse,0.0637820922947809
7,randomForest,130,1,200,friedman_mse,0.06361792859782622
8,randomForest,60,1,150,friedman_mse,0.06349228123184181
9,randomForest,210,1,200,friedman_mse,0.06371018823352528


In [ ]:
SGDregres=[]


directory='/content/gdrive/MyDrive/PatronesComportamiento/filesSGD'
files=os.listdir(directory)
for file in files:
    w=file.split('w')[1].split('f')[0]
    f=file.split('f')[1].split('.skops')[0]
    model_cv=io.load(directory+'/'+file, trusted=True)
    loss=model_cv.best_params_['loss']
    penalty=model_cv.best_params_['penalty']
    X_test_minmax=scale(singleSerieValit)
    x,y,x_test,y_test=spliting_timeseries(X_test_minmax,int(w),1) #llamo al método
    y_predicted=model_cv.predict(x_test)    
    MAE=mean_absolute_error(y_test, y_predicted)
    SGDregres.append(['SGDRegressor',w,1,loss,penalty,MAE])
    #model_cv=io.load(directory+'/'+file, trusted=True)
    
SGDregres=sorted(SGDregres, key=lambda e: e[4])

directory='/content/gdrive/MyDrive/PatronesComportamiento/filesHwideSGD'
files=os.listdir(directory)
for file in files:
    w=file.split('w')[1].split('f')[0]
    f=file.split('f')[1].split('.skops')[0]
    model_cv=io.load(directory+'/'+file, trusted=True)
    loss=model_cv.best_params_['loss']
    penalty=model_cv.best_params_['penalty']
    X_test_minmax=scale(singleSerieValit)
    x,y,x_test,y_test=spliting_timeseries(X_test_minmax,int(w),1) #llamo al método
    y_predicted=model_cv.predict(x_test)    
    MAE=mean_absolute_error(y_test, y_predicted)
    SGDregres.append(['SGDRegressor',w,f,loss,penalty,MAE])
    #model_cv=io.load(directory+'/'+file, trusted=True)
    
SGDregres=sorted(SGDregres, key=lambda e: e[4])
df2SGD = pd.DataFrame(np.array(SGDregres),
                   columns=['algoritmo','ventana','h','loss','penalty','MAE'])

/usr/local/lib/python3.9/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator SGDRegressor from version 1.1.3 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator GridSearchCV from version 1.1.3 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator SGDRegressor from version 1.1.3 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info p

In [ ]:
df2SGD

,algoritmo,ventana,h,loss,penalty,MAE
0,SGDRegressor,10,1,squared_error,elasticnet,0.019936402367042454
1,SGDRegressor,30,1,squared_error,elasticnet,0.019850139710603162
2,SGDRegressor,40,1,squared_error,elasticnet,0.01992336274754292
3,SGDRegressor,50,1,squared_error,elasticnet,0.02000900224216257
4,SGDRegressor,70,1,squared_error,elasticnet,0.019771393538549273
5,SGDRegressor,90,1,squared_error,elasticnet,0.020515855720050397
6,SGDRegressor,110,1,squared_error,elasticnet,0.02013382552249572
7,SGDRegressor,140,1,squared_error,elasticnet,0.020711116406070255
8,SGDRegressor,170,1,squared_error,elasticnet,0.02189424434323021
9,SGDRegressor,200,1,squared_error,elasticnet,0.020233728517637235
